In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

In [2]:
path = '/data/public/MLA/VCHAMPS-Test/procedures_test.csv'
procedures = pd.read_csv(path,engine = 'python',on_bad_lines='warn')
procedures.head()

Skipping line 18710187: unexpected end of data


,Unnamed: 0,Internalpatientid,Age at procedure,Procedure date,Procedure code,Procedure code description,State
0,112,100,52.559428,2014-04-19 11:18:05.0,99343,HOME VISIT FOR THE EVALUATION AND MANAGEMENT O...,New York
1,113,100,52.559428,2014-04-19 11:18:05.0,G0154,DIRECT SKILLED NURSING SERVICES OF A LICENSED ...,New York
2,114,100,52.753358,2014-06-29 08:27:24.0,97003,OCCUPATIONAL THERAPY EVALUATION,New York
3,115,100,52.753358,2014-06-29 08:27:24.0,97535,"SELF-CARE/HOME MANAGEMENT TRAINING (EG, ACTIVI...",New York
4,116,100,53.094739,2014-11-01 03:03:17.0,G0155,SERVICES OF CLINICAL SOCIAL WORKER IN HOME HEA...,New York


In [3]:
procedures['Procedure code description'].nunique()

18637

In [4]:
procedures[procedures.duplicated(subset = ['Internalpatientid', 'Procedure date','Procedure code description'],keep = False)]

,Unnamed: 0,Internalpatientid,Age at procedure,Procedure date,Procedure code,Procedure code description,State
17,129,100,55.094130,2016-10-31 21:42:18.0,99233,"SUBSEQUENT HOSPITAL CARE, PER DAY, FOR THE EVA...",New York
18,130,100,55.094130,2016-10-31 21:42:18.0,99233,"SUBSEQUENT HOSPITAL CARE, PER DAY, FOR THE EVA...",New York
19,131,100,55.103925,2016-11-04 11:38:04.0,74000,"RADIOLOGIC EXAMINATION, ABDOMEN; SINGLE ANTERO...",New York
20,132,100,55.103925,2016-11-04 11:38:04.0,74000,"RADIOLOGIC EXAMINATION, ABDOMEN; SINGLE ANTERO...",New York
21,133,100,55.103925,2016-11-04 11:38:04.0,71010,"RADIOLOGIC EXAMINATION, CHEST; SINGLE VIEW, FR...",New York
...,...,...,...,...,...,...,...
18710077,89978293,64272,84.520915,2009-08-13 09:28:01.0,99213,OFFICE OR OTHER OUTPATIENT VISIT FOR THE EVALU...,Connecticut
18710162,89979159,64636,67.238708,2005-12-21 17:56:33.0,82043,"ALBUMIN; URINE (EG, MICROALBUMIN), QUANTITATIVE",Arkansas
18710166,89979163,64636,67.238708,2005-12-21 17:56:33.0,82043,"ALBUMIN; URINE (EG, MICROALBUMIN), QUANTITATIVE",Arkansas
18710178,89979358,64640,77.234599,2008-01-11 05:54:07.0,90658,"INFLUENZA VIRUS VACCINE, TRIVALENT (IIV3), SPL...",New Jersey


In [3]:
procedures = procedures.drop_duplicates(subset = ['Internalpatientid', 'Procedure date','Procedure code description'],keep = 'first')

In [5]:
path = '/home/hassan/lily/MLA/FDA/inpatient_simple_test.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.head()

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_readmission,Died,AO,CVD,unique_admitting_specialty,...,stay_std,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,next_readmission_time,Discharge date,readmission within 300 days
0,7,5,44.40,3,0,0,0,1,0,4,...,11.420188,0.50,2,1,0,2,0,338.72,2013-04-18 14:41:31.0,0
1,9,4,10.70,2,0,0,0,0,0,3,...,1.485003,4.00,0,0,0,4,0,2.03,2000-04-10 02:38:04.0,1
2,12,5,21.54,1,0,3,0,1,1,2,...,3.648043,1.25,5,0,0,0,0,69.25,2004-04-07 03:40:16.0,1
3,17,12,46.45,2,0,4,0,0,1,4,...,3.078313,3.00,0,1,0,11,0,61.89,2004-05-05 15:18:05.0,1
4,22,2,4.53,2,1,0,0,0,0,2,...,1.831407,0.17,0,0,0,2,0,0.42,2014-05-07 15:43:10.0,1


In [9]:
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date

,Internalpatientid,Discharge date
0,7,2013-04-18 14:41:31.0
1,9,2000-04-10 02:38:04.0
2,12,2004-04-07 03:40:16.0
3,17,2004-05-05 15:18:05.0
4,22,2014-05-07 15:43:10.0
...,...,...
22250,168995,2010-06-09 10:03:21.0
22251,169011,2004-07-15 15:11:27.0
22252,169037,2015-02-12 00:48:40.0
22253,169059,2003-03-11 12:21:08.0


In [10]:
# Filter procedure date that's before discharge date
inpatient_pro = procedures.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
inpatient_pro2 = inpatient_pro.loc[inpatient_pro['Procedure date'] <= inpatient_pro['Discharge date']]
inpatient_pro2 

,Unnamed: 0,Internalpatientid,Age at procedure,Procedure date,Procedure code,Procedure code description,State,Discharge date
0,112,100,52.559428,2014-04-19 11:18:05.0,99343,HOME VISIT FOR THE EVALUATION AND MANAGEMENT O...,New York,2016-12-23 19:42:05.0
1,113,100,52.559428,2014-04-19 11:18:05.0,G0154,DIRECT SKILLED NURSING SERVICES OF A LICENSED ...,New York,2016-12-23 19:42:05.0
2,114,100,52.753358,2014-06-29 08:27:24.0,97003,OCCUPATIONAL THERAPY EVALUATION,New York,2016-12-23 19:42:05.0
3,115,100,52.753358,2014-06-29 08:27:24.0,97535,"SELF-CARE/HOME MANAGEMENT TRAINING (EG, ACTIVI...",New York,2016-12-23 19:42:05.0
4,116,100,53.094739,2014-11-01 03:03:17.0,G0155,SERVICES OF CLINICAL SOCIAL WORKER IN HOME HEA...,New York,2016-12-23 19:42:05.0
...,...,...,...,...,...,...,...,...
18034920,89979166,64636,68.825689,2007-07-24 18:56:33.0,99214,OFFICE OR OTHER OUTPATIENT VISIT FOR THE EVALU...,Arkansas,2008-05-22 04:36:31.0
18034921,89979167,64636,69.450028,2008-03-08 23:38:38.0,99213,OFFICE OR OTHER OUTPATIENT VISIT FOR THE EVALU...,Arkansas,2008-05-22 04:36:31.0
18034922,89979168,64636,69.509605,2008-03-30 18:15:06.0,92593,HEARING AID CHECK; BINAURAL,Arkansas,2008-05-22 04:36:31.0
18034923,89979169,64636,69.509605,2008-03-30 18:15:06.0,V5014,REPAIR/MODIFICATION OF A HEARING AID,Arkansas,2008-05-22 04:36:31.0


In [11]:
len(list(set(inpatient_pro2['Internalpatientid'].unique()).intersection(set(inpatient['Internalpatientid'].unique()))))

17150

### Add New features: 
Number of procedures

Number of surgery procedures

Minimum number of surgery procedures

Maximum number of surgery procedures

In [9]:
surgery_pro = procedures.loc[procedures['Procedure code description'].str.contains('SURG')]
surgery_pro['num_surgery_pro'] = surgery_pro['Procedure code'].groupby(surgery_pro['Internalpatientid']).transform('count')

/tmp/ipykernel_216312/3717331933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surgery_pro['num_surgery_pro'] = surgery_pro['Procedure code'].groupby(surgery_pro['Internalpatientid']).transform('count')


In [10]:
# Calculate total procedures before last admission for each patient
inpatient_pro2 ['total_procedure'] = inpatient_pro2['Procedure code description'].groupby(inpatient_pro2['Internalpatientid']).transform('count')

/tmp/ipykernel_216312/1437103306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_pro2 ['total_procedure'] = inpatient_pro2['Procedure code description'].groupby(inpatient_pro2['Internalpatientid']).transform('count')


In [11]:
columns = ['Internalpatientid', 'total_procedure']
df = inpatient_pro2[columns]
df = df.groupby('Internalpatientid').mean().reset_index(drop = False)
df = df.merge(surgery_pro[['Internalpatientid','num_surgery_pro']], how = 'left',on = 'Internalpatientid')
df['num_surgery_pro'] = df['num_surgery_pro'].fillna(0)
df = df.groupby('Internalpatientid').mean().reset_index(drop = False)
df

,Internalpatientid,total_procedure,num_surgery_pro
0,12,207.0,0.0
1,17,312.0,1.0
2,22,231.0,0.0
3,25,671.0,9.0
4,31,208.0,0.0
...,...,...,...
17145,168995,295.0,9.0
17146,169011,150.0,5.0
17147,169037,569.0,0.0
17148,169059,61.0,0.0


In [12]:
#df.to_csv('/home/daisy/FDA_Dataset/procedures_test_clean.csv')